## The purpose of the Capstone 1 project is to provide a list of recommendation to a gamer so that they may find new games that will interest them.

"SVDpp Collaborative game recommendation" will use SVD++ algorithm to provide a list of reommendation for people who are new to the website and therefore has no prior data on the purchase or playtime

Cleaned game review data is imported from the pickle file

In [1]:
import pickle
file_name='cleaned_review_w_language.p'
loaded_df=pickle.load(open(file_name,"rb"))
steamapp_dict=pickle.load(open("steam.p","rb"))

In [2]:
loaded_df.head()

,author,review,game_name,detected_lang,review_sentiment,recommend
0,76561198148038447,Fun! Recommend to purchase.,HalloweenStory,en,3,True
1,76561198232717920,HalloweenStory is an absolutely lovely game. A...,HalloweenStory,en,3,True
2,76561198298049420,This game is a very 2d original game simple an...,HalloweenStory,en,2,True
3,76561198893127406,I had a lot of fun playing this game =),HalloweenStory,en,2,True
4,76561198071547702,Not bad at all.\nGameplay is great.,HalloweenStory,en,3,True


In [3]:
loaded_df=loaded_df.drop_duplicates()

SVD++ algorithm from "surprise" library requires rating system to be in place. We will use the sentiment value acquired from data cleaning process for this purpose.

In [4]:
from surprise import *

In [5]:
# Surprise takes user, item, and rating as input
df=loaded_df[['author','game_name','review_sentiment']]

In [6]:
# Only take games that have more than 5 reviews into consideration 
min_games_review = 5
filter_games = df['game_name'].value_counts() > min_games_review
filter_games = filter_games[filter_games].index.tolist()

# Only take users that left more than 5 reviews into consideration 
min_user_review = 5
filter_users = df['author'].value_counts() > min_user_review
filter_users = filter_users[filter_users].index.tolist()

df_new = df[(df['game_name'].isin(filter_games)) & (df['author'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(80774, 3)
The new data frame shape:	(5556, 3)


In [7]:
# Make the reader and data form
reader = Reader(rating_scale=(0, 4))
data = Dataset.load_from_df(df_new, reader)

In [8]:
from surprise.model_selection import cross_validate
svdmodel=SVDpp()
cross_validate(svdmodel,data,measures=['RMSE'],cv=5,verbose=False)

{'test_rmse': array([0.47374817, 0.47048946, 0.45769192, 0.46365704, 0.45702226]),
 'fit_time': (1.6707384586334229,
  1.6787152290344238,
  1.646404504776001,
  1.6372699737548828,
  1.5959398746490479),
 'test_time': (0.034227609634399414,
  0.0342864990234375,
  0.03055548667907715,
  0.03358936309814453,
  0.03349947929382324)}

In [9]:
from surprise.model_selection import cross_validate
slopemodel=SVD()
cross_validate(slopemodel,data,measures=['RMSE'],cv=5,verbose=False)

{'test_rmse': array([0.4763701 , 0.45797393, 0.45855586, 0.48636563, 0.47880639]),
 'fit_time': (0.5190539360046387,
  0.5268411636352539,
  0.5162832736968994,
  0.5177481174468994,
  0.5273239612579346),
 'test_time': (0.012238264083862305,
  0.012729406356811523,
  0.011989116668701172,
  0.012176513671875,
  0.012270689010620117)}

In [10]:
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.3)

In [11]:
from surprise import accuracy
model=svdmodel.fit(trainset)
predict1=model.test(testset)
accuracy.rmse(predict1)

RMSE: 0.4631


0.46313177628596003

In [12]:
import pandas as pd
predict_df=pd.DataFrame(predict1)
predict_df['err'] = abs(predict_df.est - predict_df.r_ui)
best_predictions = predict_df.sort_values(by='err')[:10]
worst_predictions = predict_df.sort_values(by='err')[-10:]

In [13]:
best_predictions

,uid,iid,r_ui,est,details,err
1055,76561198025825159,Z69,2.0,1.999848,{'was_impossible': False},0.000152
962,76561198066590240,Rift Coaster HD Remastered VR,2.0,2.000162,{'was_impossible': False},0.000162
18,76561198019388720,DeerHunterX,2.0,2.000180,{'was_impossible': False},0.000180
1270,76561198028673669,Mad Arkanoid,2.0,1.999740,{'was_impossible': False},0.000260
748,76561197999151078,War Operations,2.0,2.000480,{'was_impossible': False},0.000480
106,76561198094803808,Dungeon Defenders Halloween Costume Pack,2.0,1.999302,{'was_impossible': False},0.000698
895,76561198139328230,Pyrite Heart,2.0,2.000753,{'was_impossible': False},0.000753
1313,76561198137285867,ShapeShifter,2.0,2.000803,{'was_impossible': False},0.000803
1017,76561197969752619,The Flying Dutchman,2.0,2.000882,{'was_impossible': False},0.000882
1451,76561197969752619,Cranky Cat,2.0,2.000882,{'was_impossible': False},0.000882


In [14]:
worst_predictions

,uid,iid,r_ui,est,details,err
225,76561197971880699,Panzer Corps: Afrika Korps,4.0,2.269766,{'was_impossible': False},1.730234
1039,76561198254317898,The Grand Canyon VR Experience,4.0,2.235387,{'was_impossible': False},1.764613
1249,76561198135143088,Chicken Labyrinth Puzzles,4.0,2.199966,{'was_impossible': False},1.800034
1428,76561198165965131,Hyperdimension Neptunia Re;Birth3 Deluxe Pack,4.0,2.178301,{'was_impossible': False},1.821699
1305,76561198125100728,NARUTO SHIPPUDEN: Ultimate Ninja STORM Revolut...,4.0,2.114523,{'was_impossible': False},1.885477
1057,76561198172055451,Time Clickers,4.0,2.104951,{'was_impossible': False},1.895049
1611,76561198381798216,Tactical Soccer The New Season,4.0,2.097877,{'was_impossible': False},1.902123
1202,76561198067136654,Mercury: Cascade into Madness,4.0,2.014696,{'was_impossible': False},1.985304
1331,76561197965049785,Total War: ROME II - Greek States Culture Pack,0.0,2.053866,{'was_impossible': False},2.053866
1417,76561197982288410,Mad-Sector,4.0,1.899756,{'was_impossible': False},2.100244


In [15]:
test_id='76561198141485825'
pred=model.predict(uid=test_id,iid="Fallout 4")
score=pred.est

In [16]:
import requests
# Get all the games that the player played
def getownedgames(idnum):
    """Takes the id number of a steam user and returns the id num of games they own"""
    url='http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=3A629E52AE594CB777253431189F2967&steamid={}&format=json'.format(idnum)
    r=requests.get(url)
    json_data=r.json()
    return json_data

In [17]:
def rawplayergamelist(_id):
    gamelist=getownedgames(_id)['response']['games']
    return gamelist

In [18]:
testperson=rawplayergamelist(test_id)

In [19]:
testids=list()
testplaytimes=list()
for i in range(len(testperson)):
    testids.append(testperson[i]['appid'])
    testplaytimes.append(testperson[i]['playtime_forever'])

In [20]:
import pandas as pd

In [21]:
test_df=pd.DataFrame(list(zip(testids,testplaytimes)),columns=['ids','playtime']).sort_values('playtime',ascending=False)
playedgame_name=list()
for id_ in test_df['ids']:
    name=steamapp_dict.get(id_)
    playedgame_name.append(name)
test_df['name']=playedgame_name

In [22]:
tests=[[test_id,iid,5] for iid in predict_df["iid"].unique().tolist()]
predictions=model.test(tests)
pred_df=pd.DataFrame(predictions)

In [23]:
pred_df.head(10)

,uid,iid,r_ui,est,details
0,76561198141485825,Hopalong: The Badlands,5,2.125031,{'was_impossible': False}
1,76561198141485825,Call of Cthulhu: The Wasted Land,5,2.144767,{'was_impossible': False}
2,76561198141485825,Soldiers of the Universe,5,2.144767,{'was_impossible': False}
3,76561198141485825,Winged Sakura: Mindy's Arc,5,2.022337,{'was_impossible': False}
4,76561198141485825,OneManVurgeR,5,2.127666,{'was_impossible': False}
5,76561198141485825,Bloop,5,2.144767,{'was_impossible': False}
6,76561198141485825,Tree of Life - Upgrade to Adventurer,5,2.144767,{'was_impossible': False}
7,76561198141485825,Fallen Legion+,5,2.144767,{'was_impossible': False}
8,76561198141485825,Command HQ,5,2.144767,{'was_impossible': False}
9,76561198141485825,Mythos: The Beginning,5,2.144767,{'was_impossible': False}


In [24]:
test_df=pd.DataFrame(list(zip(testids,testplaytimes)),columns=['ids','playtime']).sort_values('playtime',ascending=False)
playedgame_name=list()
for id_ in test_df['ids']:
    name=steamapp_dict.get(id_)
    playedgame_name.append(name)

In [25]:
removed=[name for name in pred_df['iid'].tolist() if name not in playedgame_name]

In [26]:
pred_df=pred_df.set_index(['iid'])

In [27]:
finaltop10recommendation=pred_df.loc[removed].sort_values(['est'],ascending=False)[:10]

In [28]:
pred_game=finaltop10recommendation.index[0]
pred_rate=finaltop10recommendation['est'][0]
print('Top item for user {0} is {1} with predicted rating {2}'.format(test_id,pred_game, pred_rate))

Top item for user 76561198141485825 is Hiveswap Friendsim - Volume Three with predicted rating 2.488845812482942


Improving the model

In [29]:
from surprise.model_selection import GridSearchCV
param_grid={'lr_all' : [0.0001,.001,.01,.1], 'reg_all':[0.025,.05,.1,.5,.8]}
gs=GridSearchCV(SVDpp,param_grid,measures=['rmse','mae'],cv=3)
gs.fit(data)
print(gs.best_params['rmse']['lr_all'],gs.best_params['rmse']['reg_all'])

0.01 0.5


In [30]:
gs.best_params

{'rmse': {'lr_all': 0.01, 'reg_all': 0.5},
 'mae': {'lr_all': 0.01, 'reg_all': 0.5}}

In [31]:
new_model=SVDpp(lr_all=gs.best_params['rmse']['lr_all'],
                reg_all=gs.best_params['rmse']['reg_all'])
cross_validate(new_model,data,verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4578  0.4498  0.4417  0.4892  0.4819  0.4641  0.0184  
MAE (testset)     0.2940  0.2877  0.2865  0.3107  0.3138  0.2985  0.0115  
Fit time          1.65    1.62    1.65    1.71    1.70    1.67    0.03    
Test time         0.03    0.03    0.03    0.04    0.03    0.03    0.00    


{'test_rmse': array([0.45776181, 0.4498388 , 0.44165264, 0.48924025, 0.481875  ]),
 'test_mae': array([0.29400739, 0.28771382, 0.28653406, 0.31065129, 0.31376607]),
 'fit_time': (1.6495184898376465,
  1.622614860534668,
  1.6496880054473877,
  1.70644211769104,
  1.697782039642334),
 'test_time': (0.03345894813537598,
  0.03318023681640625,
  0.03367424011230469,
  0.03507041931152344,
  0.03222775459289551)}

In [32]:
from surprise.model_selection import train_test_split
trainset,testset=train_test_split(data,test_size=0.3)

In [33]:
from surprise import accuracy
model2=new_model.fit(trainset)
predict2=model2.test(testset)
accuracy.rmse(predict2)

RMSE: 0.4938


0.4937689514411917

In [34]:
import pandas as pd
predict_df2=pd.DataFrame(predict2)
predict_df2['err'] = abs(predict_df2.est - predict_df2.r_ui)
best_predictions2 = predict_df2.sort_values(by='err')[:10]
worst_predictions2 = predict_df2.sort_values(by='err')[-10:]

In [35]:
best_predictions2

,uid,iid,r_ui,est,details,err
1238,76561197964604365,→SUMETRICK←,2.0,2.000061,{'was_impossible': False},0.000061
392,76561198000561257,Strategy & Tactics: Dark Ages,2.0,2.000176,{'was_impossible': False},0.000176
714,76561198005439390,Hyperdimension Neptunia Re;Birth1 New Content ...,2.0,1.999739,{'was_impossible': False},0.000261
43,76561198025037510,Castle Invasion: Throne Out,2.0,2.000364,{'was_impossible': False},0.000364
770,76561198025037510,Leave Me Alone: A Trip To Hell,2.0,2.000364,{'was_impossible': False},0.000364
598,76561198066590240,Lost in the Rift - Reborn,2.0,2.000445,{'was_impossible': False},0.000445
1032,76561197982837520,The Forest of Doom,2.0,2.000849,{'was_impossible': False},0.000849
1319,76561197991880022,Bloody Mice,2.0,2.000991,{'was_impossible': False},0.000991
1598,76561198049701098,In Fear I Trust,2.0,2.001315,{'was_impossible': False},0.001315
831,76561198138437975,The Way We ALL GO,2.0,1.998616,{'was_impossible': False},0.001384


In [36]:
worst_predictions2

,uid,iid,r_ui,est,details,err
490,76561198141837298,Sniper Elite 4 - Deathstorm Part 2: Infiltration,4.0,2.114360,{'was_impossible': False},1.885640
500,76561198005539606,Escape Room VR: Stories,4.0,2.113402,{'was_impossible': False},1.886598
1457,76561198381798216,Tactical Soccer The New Season,4.0,2.099974,{'was_impossible': False},1.900026
1252,76561198172055451,Time Clickers,4.0,2.072852,{'was_impossible': False},1.927148
262,76561197982288410,Mad-Sector,4.0,2.071220,{'was_impossible': False},1.928780
705,76561198019816374,Eruption,4.0,2.055082,{'was_impossible': False},1.944918
1003,76561197965049785,Total War: ROME II - Greek States Culture Pack,0.0,2.073927,{'was_impossible': False},2.073927
1616,76561198034766576,Hero Siege - Doom Bringer (Skin),0.0,2.077333,{'was_impossible': False},2.077333
40,76561198888079295,21+,0.0,2.209149,{'was_impossible': False},2.209149
97,76561198059971262,Awesomenauts - Coco Nebulon Announcer,0.0,2.218884,{'was_impossible': False},2.218884


The best predictions got a little worse but the worst predictions got a little better.

In [37]:
tests=[[test_id,iid,5] for iid in predict_df2["iid"].unique().tolist()]
new_predictions=new_model.test(tests)
pred_df2=pd.DataFrame(new_predictions)

In [38]:
pred_df2=pred_df2.set_index(['iid'])
new_top10recommendation=pred_df2.loc[removed].sort_values(['est'],ascending=False)[:10]

/home/seo/environments/my_env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [39]:
pred_game2=new_top10recommendation.index[0]
pred_rate2=new_top10recommendation['est'][0]
print('Top item for user {0} is {1} with predicted rating {2}'.format(test_id,pred_game2, pred_rate2))

Top item for user 76561198141485825 is Hiveswap Friendsim - Volume Three with predicted rating 2.557066566217302


In [40]:
top10=pred_df2.sort_values(['est'],ascending=False)[:10]
recommended_games=list()
for name in top10.index.tolist():
    ids=list(steamapp_dict.keys())[list(steamapp_dict.values()).index(name)]
    recommended_games.append((name,"https://store.steampowered.com/app/"+str(ids)))

In [41]:
top10

,uid,r_ui,est,details
iid,,,,
Hiveswap Friendsim - Volume Three,76561198141485825,5,2.557067,{'was_impossible': False}
Train Simulator: The Riviera Line: Exeter - Paignton,76561198141485825,5,2.518785,{'was_impossible': False}
Awesomenauts - Abyssal Swiggins Skin,76561198141485825,5,2.463186,{'was_impossible': False}
Panzer Corps: Allied Corps,76561198141485825,5,2.461297,{'was_impossible': False}
Digital Domain’s Monkey King™,76561198141485825,5,2.459878,{'was_impossible': False}
Civilization V - Scrambled Nations Map Pack,76561198141485825,5,2.447839,{'was_impossible': False}
Tomb Raider: Demolition,76561198141485825,5,2.446313,{'was_impossible': False}
Train Simulator: West Highland Line Extension Route Add-On,76561198141485825,5,2.444723,{'was_impossible': False}
Demonlisher,76561198141485825,5,2.443932,{'was_impossible': False}


In [42]:
print("Based on what other players are playing a lot, here are some of the games you may like.")
print(pd.DataFrame(recommended_games,columns=["Game Name","Store page"]))

Based on what other players are playing a lot, here are some of the games you may like.
                                           Game Name  \
0                  Hiveswap Friendsim - Volume Three   
1  Train Simulator: The Riviera Line: Exeter - Pa...   
2               Awesomenauts - Abyssal Swiggins Skin   
3                         Panzer Corps: Allied Corps   
4                      Digital Domain’s Monkey King™   
5        Civilization V - Scrambled Nations Map Pack   
6                            Tomb Raider: Demolition   
7  Train Simulator: West Highland Line Extension ...   
8                                        Demonlisher   
9                                            GTGD S2   

                                  Store page  
0  https://store.steampowered.com/app/859130  
1  https://store.steampowered.com/app/222632  
2  https://store.steampowered.com/app/251080  
3  https://store.steampowered.com/app/276552  
4  https://store.steampowered.com/app/686030  
5  https://st